In [1]:
import os
import re
from collections import defaultdict
import pandas as pd

In [3]:
TOTAL = True

In [4]:
directory = "../data"

if not TOTAL:
    directory = "../pre/sample"

In [8]:
!rm -fr ../pre/counts/*

In [5]:
def format_text(text : str):
    text = text.lower()
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'[^\w0-9- ]+', '', text, flags=re.UNICODE)
    
    return text


def count_words(file_name):
    word_count = defaultdict(int)
    
    with open(file_name, 'r') as f:
        text = f.read()
        text = format_text(text)

        text = [x for x in text.split() if len(x) > 0]

        for word in text:
            word_count[word] += 1
        

    return word_count


def count_words_pd (file_name):
    word_count_dict = count_words(file_name)

    # Getting the count dataframe
    word_count_df = pd.DataFrame(word_count_dict.items(), columns=['word', 'count'])
    
    # sorting by count
    word_count_df.sort_values(by='count', ascending=False, inplace=True)
    
    return word_count_df


def build_counts(directory):
    directory = "../pre/sample"
    
    data_list = os.listdir(directory)
    data_list = list(data_list)
    
    os.makedirs('../pre/counts', exist_ok=True)
        
    for file in data_list:
        # Remove the extension using 
        filename = os.path.splitext(file)[0]
        
        # Count the words
        count = count_words_pd(f'{directory}/{file}')
            
        # save as csv
        count.to_csv(f'../pre/counts/{filename}.csv', index=False)
    
    
def build_total_counts():
    data_list = os.listdir('../pre/counts')
    data_list = list(data_list)
    
    os.makedirs('../pre/global', exist_ok=True)
    
    glob_count = defaultdict(int)
    
    for file in data_list:
        count = pd.read_csv(f'../pre/counts/{file}')
        count_dict = dict(zip(count['word'], count['count']))
        
        for word, count in count_dict.items():
            glob_count[word] += count
        
    glob_count = pd.DataFrame(glob_count.items(), columns=['word', 'count'])
    glob_count.sort_values(by='count', ascending=False, inplace=True)
    
    glob_count.to_csv(f'../pre/global/counts.csv', index=False)

In [6]:
build_counts(directory)

In [7]:
build_total_counts()